In [1]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (41.0.1)


In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [3]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

W0705 15:10:23.739706 139864494737280 abstractdisplay.py:151] xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    print(mp4)
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', video_callable=lambda episode_id: episode_id%50==0 ,force=True)
  return env

In [0]:
def to_grayscale(img):
    return np.mean(img, axis=2).astype(np.uint8)

def downsample(img):
    return img[::2, ::2]

def preprocess(img):
    return to_grayscale(downsample(img))

def transform_reward(reward):
        return np.sign(reward)

In [0]:
import random
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd

In [0]:
class MemoryBuffer:
	def __init__(self,capacity=10000,obs_shape=(84,84,4)):
		self.capacity = capacity
		self.data = {
			'state': np.empty(shape=(self.capacity,obs_shape[0],obs_shape[1],obs_shape[2]),dtype=np.uint8),
			'action': np.empty(shape=(self.capacity,1),dtype=np.uint8),
			'reward': np.empty(shape=(self.capacity,1),dtype=np.int8),
			'next_state': np.empty(shape=(self.capacity,obs_shape[0],obs_shape[1],obs_shape[2]),dtype=np.uint8),
			'done': np.empty(shape=(self.capacity,1),dtype=np.bool)
		}

		self.meta_data = {
			'buffer_ptr': 0,
			'fill_size': 0
		}

	def add(self,state,action,reward,next_state,done):
		idx = self.meta_data['buffer_ptr']
		fill_size = self.meta_data['fill_size']

		self.data['state'][idx] = state
		self.data['action'][idx] = action
		self.data['reward'][idx] = reward
		self.data['next_state'][idx] = next_state
		self.data['done'][idx] = done

		idx += 1
		fill_size = max(fill_size,idx)
		idx = idx % self.capacity

		self.meta_data['buffer_ptr'] = idx
		self.meta_data['fill_size'] = fill_size

	def get_minibatch(self,batch_size=32):
		sample_idx = np.random.choice(self.meta_data['fill_size'],batch_size,replace=False)
		state_batch = self.data['state'][sample_idx,...]
		action_batch = self.data['action'][sample_idx,...]
		reward_batch = self.data['reward'][sample_idx,...]
		next_state_batch = self.data['next_state'][sample_idx,...]
		done_batch = self.data['done'][sample_idx,...]

		return state_batch,action_batch,reward_batch,next_state_batch,done_batch


In [0]:
import numpy as np
import os
os.environ.setdefault('PATH', '')
from collections import deque
import gym
from gym import spaces
import cv2
cv2.ocl.setUseOpenCL(False)

class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.randint(1, self.noop_max + 1) #pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done  = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condition for a few frames
            # so it's important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip       = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)

class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env, width=84, height=84, grayscale=True):
        """Warp frames to 84x84 as done in the Nature paper and later work."""
        gym.ObservationWrapper.__init__(self, env)
        self.width = width
        self.height = height
        self.grayscale = grayscale
        if self.grayscale:
            self.observation_space = spaces.Box(low=0, high=255,
                shape=(self.height, self.width, 1), dtype=np.uint8)
        else:
            self.observation_space = spaces.Box(low=0, high=255,
                shape=(self.height, self.width, 3), dtype=np.uint8)

    def observation(self, frame):
        if self.grayscale:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        if self.grayscale:
            frame = np.expand_dims(frame, -1)
        return frame

class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[:-1] + (shp[-1] * k,)), dtype=env.observation_space.dtype)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))

class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=env.observation_space.shape, dtype=np.float32)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0

class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.
        This object should only be converted to numpy array before being passed to the model.
        You'd not believe how complex the previous solution was."""
        self._frames = frames
        self._out = None

    def _force(self):
        if self._out is None:
            self._out = np.concatenate(self._frames, axis=-1)
            self._frames = None
        return self._out

    def __array__(self, dtype=None):
        out = self._force()
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._force())

    def __getitem__(self, i):
        return self._force()[i]

def make_atari(env_id, timelimit=True):
    # XXX(john): remove timelimit argument after gym is upgraded to allow double wrapping
    env = gym.make(env_id)
    if not timelimit:
        env = env.env
    assert 'NoFrameskip' in env.spec.id
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip=4)
    return env

def wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=False, scale=False):
    """Configure environment for DeepMind-style Atari.
    """
    if episode_life:
        env = EpisodicLifeEnv(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = WarpFrame(env)
    if scale:
        env = ScaledFloatFrame(env)
    if clip_rewards:
        env = ClipRewardEnv(env)
    if frame_stack:
        env = FrameStack(env, 4)
    return env

In [0]:
class NeuralNet:
  def __init__(self,input_shape,num_actions,learning_rate=0.00025):
    frames=keras.layers.Input(input_shape,name='frames')
    normalize=keras.layers.Lambda(lambda x : x/255.0)(frames)
    conv_1 = keras.layers.Conv2D(32, (8, 8), activation="relu", strides=(4, 4))(normalize)
    conv_2 = keras.layers.Conv2D(64, (4, 4), activation="relu", strides=(2, 2))(conv_1)
    conv_3 = keras.layers.Conv2D(64, (3, 3), activation="relu", strides=(1, 1))(conv_2)
    conv_flattened = keras.layers.Flatten()(conv_3)
    hidden = keras.layers.Dense(512, activation='relu')(conv_flattened)
    output = keras.layers.Dense(num_actions)(hidden)
    model = keras.models.Model(inputs=frames, outputs=output)
    optimizer = optimizer=keras.optimizers.Adam(lr=learning_rate)#, rho=0.95, epsilon=0.01)
    model.compile(optimizer,loss=self.huber_loss)
    self.model=model
  def huber_loss(self,y_true, y_pred):
    return tf.losses.huber_loss(y_true,y_pred)

In [0]:
class LinearSchedule:
	def __init__(self,schedule_timesteps,final_p,initial_p=1.0):
		self.schedule_timesteps = schedule_timesteps
		self.final_p = final_p
		self.initial_p = initial_p
	def value(self,t):
		fraction = min(float(t)/self.schedule_timesteps , 1.0)
		return self.initial_p + fraction * (self.final_p - self.initial_p)

In [0]:
class Agent:
  def __init__(self,env,gamma=0.99,batch_size=32,learning_rate=0.00025,learn_start=10000,buffer_size=10000,update_freq=1000,train_freq=4,epsilon_min=0.01,exploration_fraction=0.1,tot_steps=int(1e7)):
    self.env=env
    self.gamma=gamma
    self.learning_rate=learning_rate
    self.capacity=buffer_size
    self.update_freq=update_freq
    self.train_freq=train_freq
    self.epsilon_min = epsilon_min
    self.exploration_fraction = exploration_fraction
    self.tot_steps = tot_steps
    self.epsilon = 1.0
    self.input_shape=env.observation_space.shape
    self.batch_size=batch_size
    self.num_actions=env.action_space.n
    self.learn_start=learn_start
    self.exploration = LinearSchedule(
      schedule_timesteps=int(self.exploration_fraction * self.tot_steps),
      initial_p=self.epsilon,
      final_p=self.epsilon_min
    )
    
    self.replay_buffer=MemoryBuffer(self.capacity,self.input_shape)
    self.target_model=NeuralNet(self.input_shape,self.num_actions,self.learning_rate).model
    self.local_model=NeuralNet(self.input_shape,self.num_actions,self.learning_rate).model
    self.update_target()
    
     
  def update_target(self):
    self.target_model.set_weights(self.local_model.get_weights()) 

  def update_exploration(self,t):
    self.epsilon = self.exploration.value(t)

  def remember(self,state,action,rew,new_state,done):
    self.replay_buffer.add(obs,action,rew,new_obs,done)

  def act(self,obs):
    if np.random.rand() < self.epsilon:
      return self.env.action_space.sample()
    q_vals = self.local_model.predict(np.expand_dims(obs,axis=0),batch_size=1)
    return np.argmax(q_vals[0])

  def learn(self):
    if self.replay_buffer.meta_data['fill_size'] < self.batch_size:
      return
    curr_obs,action,reward,next_obs,done = self.replay_buffer.get_minibatch(self.batch_size)
    target = self.local_model.predict(curr_obs,batch_size=self.batch_size)
    done_mask = done.ravel()
    undone_mask = np.invert(done).ravel()
    target[done_mask,action[done_mask].ravel()] = reward[done_mask].ravel()
    Q_target = self.target_model.predict(next_obs,batch_size=self.batch_size)
    Q_future = np.max(Q_target[undone_mask],axis=1)
    target[undone_mask,action[undone_mask].ravel()] = reward[undone_mask].ravel() + self.gamma * Q_future
    self.local_model.fit(curr_obs, target, batch_size=self.batch_size, verbose=0)

  def step_update(self,t):
    if t<=self.learn_start:
      return
    if t%self.train_freq==0:
      self.learn()
    if t%self.update_freq==0:
      self.update_target()


In [0]:
import tensorflow as tf
import tensorflow.keras as keras

In [0]:
env=make_atari("PongNoFrameskip-v4")
env=wrap_deepmind(env,frame_stack=True,scale=False)
env=wrap_env(env)

In [14]:
agent=Agent(env,learning_rate=0.0001)

W0705 15:10:48.779996 139864494737280 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0705 15:10:48.916182 139864494737280 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
t=0
obs=env.reset().__array__(dtype=np.uint8)
episode=0
re=0

In [34]:
while t<=agent.tot_steps:
  action=agent.act(obs)
  new_obs,reward,done,_=env.step(action)
  re+=reward
  agent.remember(obs,action,reward,new_obs,done)
  obs=new_obs
  if done:
    obs=env.reset().__array__(dtype=np.uint8)
    print("Reward at Episode",episode,'is',re)
    re=0
    if episode%50==0:
      show_video()
    episode+=1
  agent.step_update(t)
  agent.update_exploration(t)
  t+=1

Reward at Episode 600 is -11.0
video/openaigym.video.0.128.video000000.mp4


Reward at Episode 601 is -2.0
Reward at Episode 602 is -6.0
Reward at Episode 603 is -5.0
Reward at Episode 604 is -4.0
Reward at Episode 605 is -12.0
Reward at Episode 606 is -3.0
Reward at Episode 607 is 2.0
Reward at Episode 608 is 10.0
Reward at Episode 609 is 9.0
Reward at Episode 610 is 5.0
Reward at Episode 611 is 9.0
Reward at Episode 612 is 9.0
Reward at Episode 613 is 9.0
Reward at Episode 614 is 11.0
Reward at Episode 615 is 8.0
Reward at Episode 616 is -11.0
Reward at Episode 617 is -15.0
Reward at Episode 618 is -8.0
Reward at Episode 619 is -5.0
Reward at Episode 620 is 11.0
Reward at Episode 621 is 12.0
Reward at Episode 622 is -4.0
Reward at Episode 623 is -8.0
Reward at Episode 624 is 10.0
Reward at Episode 625 is 14.0
Reward at Episode 626 is 8.0
Reward at Episode 627 is 14.0
Reward at Episode 628 is 9.0
Reward at Episode 629 is 9.0
Reward at Episode 630 is 9.0
Reward at Episode 631 is 15.0
Reward at Episode 632 is 9.0
Reward at Episode 633 is 16.0
Reward at Episode 6

Reward at Episode 651 is 18.0
Reward at Episode 652 is 16.0
Reward at Episode 653 is 18.0
Reward at Episode 654 is 16.0
Reward at Episode 655 is 16.0
Reward at Episode 656 is 19.0
Reward at Episode 657 is 17.0
Reward at Episode 658 is 18.0
Reward at Episode 659 is 19.0
Reward at Episode 660 is 18.0
Reward at Episode 661 is 19.0
Reward at Episode 662 is 20.0
Reward at Episode 663 is 20.0
Reward at Episode 664 is 19.0
Reward at Episode 665 is 20.0
Reward at Episode 666 is 20.0
Reward at Episode 667 is 19.0
Reward at Episode 668 is 20.0
Reward at Episode 669 is 20.0
Reward at Episode 670 is 19.0
Reward at Episode 671 is 20.0
Reward at Episode 672 is 21.0
Reward at Episode 673 is 19.0
Reward at Episode 674 is 20.0
Reward at Episode 675 is 19.0
Reward at Episode 676 is 20.0
Reward at Episode 677 is 21.0
Reward at Episode 678 is 19.0
Reward at Episode 679 is 21.0
Reward at Episode 680 is 21.0
Reward at Episode 681 is 19.0
Reward at Episode 682 is 19.0
Reward at Episode 683 is 20.0
Reward at 

Reward at Episode 701 is 19.0
Reward at Episode 702 is 20.0
Reward at Episode 703 is 19.0
Reward at Episode 704 is 20.0
Reward at Episode 705 is 19.0
Reward at Episode 706 is 20.0
Reward at Episode 707 is 19.0
Reward at Episode 708 is 18.0
Reward at Episode 709 is 20.0
Reward at Episode 710 is 19.0
Reward at Episode 711 is 20.0
Reward at Episode 712 is 21.0
Reward at Episode 713 is 20.0
Reward at Episode 714 is 20.0
Reward at Episode 715 is 21.0
Reward at Episode 716 is 21.0
Reward at Episode 717 is 21.0
Reward at Episode 718 is 19.0
Reward at Episode 719 is 20.0
Reward at Episode 720 is 20.0
Reward at Episode 721 is 21.0
Reward at Episode 722 is 18.0
Reward at Episode 723 is 20.0
Reward at Episode 724 is 18.0
Reward at Episode 725 is 19.0
Reward at Episode 726 is 18.0
Reward at Episode 727 is 20.0
Reward at Episode 728 is 20.0
Reward at Episode 729 is 20.0
Reward at Episode 730 is 21.0
Reward at Episode 731 is 19.0
Reward at Episode 732 is 20.0
Reward at Episode 733 is 20.0
Reward at 

Reward at Episode 751 is 19.0
Reward at Episode 752 is 20.0
Reward at Episode 753 is 20.0
Reward at Episode 754 is 18.0
Reward at Episode 755 is 20.0
Reward at Episode 756 is 20.0
Reward at Episode 757 is 20.0
Reward at Episode 758 is 20.0
Reward at Episode 759 is 20.0
Reward at Episode 760 is 19.0
Reward at Episode 761 is 20.0
Reward at Episode 762 is 19.0
Reward at Episode 763 is 21.0
Reward at Episode 764 is 21.0
Reward at Episode 765 is 20.0
Reward at Episode 766 is 21.0
Reward at Episode 767 is 19.0
Reward at Episode 768 is 19.0
Reward at Episode 769 is 19.0
Reward at Episode 770 is 21.0
Reward at Episode 771 is 21.0
Reward at Episode 772 is 18.0
Reward at Episode 773 is 21.0
Reward at Episode 774 is 20.0
Reward at Episode 775 is 19.0
Reward at Episode 776 is 21.0
Reward at Episode 777 is 21.0
Reward at Episode 778 is 21.0
Reward at Episode 779 is 17.0
Reward at Episode 780 is 19.0
Reward at Episode 781 is 19.0
Reward at Episode 782 is 19.0
Reward at Episode 783 is 21.0
Reward at 

Reward at Episode 801 is 21.0
Reward at Episode 802 is 20.0
Reward at Episode 803 is 21.0
Reward at Episode 804 is 21.0
Reward at Episode 805 is 21.0
Reward at Episode 806 is 21.0
Reward at Episode 807 is 21.0
Reward at Episode 808 is 19.0
Reward at Episode 809 is 20.0
Reward at Episode 810 is 19.0
Reward at Episode 811 is 19.0
Reward at Episode 812 is 19.0
Reward at Episode 813 is 21.0
Reward at Episode 814 is 20.0
Reward at Episode 815 is 21.0
Reward at Episode 816 is 21.0
Reward at Episode 817 is 21.0
Reward at Episode 818 is 21.0
Reward at Episode 819 is 21.0
Reward at Episode 820 is 19.0
Reward at Episode 821 is 21.0
Reward at Episode 822 is 20.0
Reward at Episode 823 is 20.0
Reward at Episode 824 is 21.0
Reward at Episode 826 is 21.0
Reward at Episode 827 is 20.0
Reward at Episode 828 is 18.0
Reward at Episode 829 is 20.0
Reward at Episode 830 is 20.0
Reward at Episode 831 is 21.0
Reward at Episode 832 is 20.0
Reward at Episode 833 is 20.0
Reward at Episode 834 is 21.0
Reward at 

Reward at Episode 851 is 21.0
Reward at Episode 852 is 20.0
Reward at Episode 853 is 21.0
Reward at Episode 854 is 19.0
Reward at Episode 855 is 20.0
Reward at Episode 856 is 20.0
Reward at Episode 857 is 21.0
Reward at Episode 858 is 18.0
Reward at Episode 859 is 21.0
Reward at Episode 860 is 20.0
Reward at Episode 861 is 21.0
Reward at Episode 862 is 20.0
Reward at Episode 863 is 19.0
Reward at Episode 864 is 20.0
Reward at Episode 865 is 19.0
Reward at Episode 866 is 20.0
Reward at Episode 867 is 20.0
Reward at Episode 868 is 19.0
Reward at Episode 869 is 20.0
Reward at Episode 870 is 21.0
Reward at Episode 871 is 20.0
Reward at Episode 872 is 20.0
Reward at Episode 873 is 21.0
Reward at Episode 874 is 21.0
Reward at Episode 875 is 21.0
Reward at Episode 876 is 20.0
Reward at Episode 877 is 21.0
Reward at Episode 878 is 21.0
Reward at Episode 879 is 21.0
Reward at Episode 880 is 18.0
Reward at Episode 881 is 20.0
Reward at Episode 882 is 14.0
Reward at Episode 883 is 21.0
Reward at 

KeyboardInterrupt: ignored

In [35]:
t

1397565

In [0]:
import pickle
data={
    'memory':agent.replay_buffer,
    't':t
}

agent.target_model.save('/content/drive/My Drive/Colab Notebooks/target_modelfinal.h5')
agent.local_model.save('/content/drive/My Drive/Colab Notebooks/local_modelfinal.h5')
with open('/content/drive/My Drive/Colab Notebooks/memoryfinal.pickle', 'wb') as handle:
    pickle.dump(data, handle)

In [16]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pickle
with open('/content/drive/My Drive/Colab Notebooks/memoryfinal.pickle', 'rb') as handle:
    data=pickle.load(handle)

In [0]:
agent.replay_buffer=data['memory']
t=data['t']

In [0]:
from tensorflow.keras.models import load_model

In [31]:
agent.local_model= load_model("/content/drive/My Drive/Colab Notebooks/local_modelfinal.h5")

W0705 15:20:08.369441 139864494737280 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0705 15:20:08.371616 139864494737280 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
agent.target_model= load_model("/content/drive/My Drive/Colab Notebooks/target_modelfinal.h5")

In [0]:
episode=800